In [1]:
import pandas as pd
from config import CONFIG
from constants import *
from read_brfss_file import ReadBRFSS
from sklearn.linear_model import LogisticRegression
import numpy as np
from get_population_by_state import GetAgeSex

In [2]:
fname = CONFIG.get('BRFSS').get('2017')

In [3]:
COLUMNS_FOR_ODD_RATIO = ["SMOKE100", "_STATE", "ASTHMA3", "ASTHNOW", "_AGEG5YR", "SEX", "_RACE_G1",
"INCOME2", "_INCOMG", "EDUCA", "_EDUCAG", "_BMI5", "_BMI5CAT"]

RENAME  = {
    "SEX1": "SEX",
    "SOMKE100": "SMOKE100"
}

In [4]:
years = CONFIG.get("analysis_years")
dfs = []
for year in years:
    fname = "data/{}/{}/{}".format("BRFSS", year, CONFIG.get('BRFSS').get(year))
    df = pd.read_sas(fname)
    for i, j in RENAME.items():
        try:
            df[j] = df[i]
        except:
            pass
    df = df[COLUMNS_FOR_ODD_RATIO]
    dfs.append(df)
    print(fname)

data/BRFSS/2016/LLCP2016.XPT
data/BRFSS/2017/LLCP2017.XPT
data/BRFSS/2018/LLCP2018.XPT


In [144]:
mdf = pd.DataFrame()
for i in [0, 1, 2]:
    mdf = mdf.append(dfs[i])

In [145]:
mdf = mdf[mdf['SMOKE100'] == 2]

In [146]:
def if_carb(x):
    if x in CARB:
        return 1
    return 0

mdf['TAILPIPE'] = mdf['_STATE'].apply(if_carb)

In [147]:
mdf['ASTHMA3'].unique()

array([2., 1., 7., 9.])

In [148]:
mdf = mdf[mdf['ASTHMA3'].isin([1,2])] 

def get_primary_risk(x):
    if x['ASTHMA3'] == 1:
        if x['ASTHNOW'] == 1:
            return 1
    return 0

mdf['ASTHMA'] = mdf.apply(get_primary_risk, axis=1)

In [149]:
# adding pop
population__df = GetAgeSex('2017').read_file_by_population()
population__df = population__df[population__df['STATE']!=0]
population__df['POPEST2017_CIV'] = (population__df['POPEST2017_CIV'] - population__df['POPEST2017_CIV'].mean()) / population__df['POPEST2017_CIV'].std()
mdf = mdf.merge(population__df[['STATE', 'POPEST2017_CIV']], left_on="_STATE", right_on="STATE", how="left")

In [150]:
# adding epa region

epa_region = pd.read_csv("data/states_and_counties.csv")
epa_region = epa_region[['State Code', "EPA Region"]]
epa_region = epa_region[epa_region['State Code']!='CC']
epa_region['State Code'] = epa_region['State Code'].apply(int)
epa_region = epa_region.drop_duplicates(['State Code'], keep='first')
mdf = mdf.merge(epa_region, left_on="_STATE", right_on="State Code", how='left')

In [152]:
MODELING_COLUMNS = ["TAILPIPE", "_AGEG5YR", "SEX", "_RACE_G1", "_INCOMG", "_EDUCAG", "_BMI5CAT", 'POPEST2017_CIV', "EPA Region", "ASTHMA"]
mdf = mdf[MODELING_COLUMNS]
mdf['POPEST2017_CIV'] = mdf['POPEST2017_CIV'].fillna(mdf['POPEST2017_CIV'].mean())
for column in mdf.columns:
    if mdf[column].isna().mean() > 0:
        mdf[column] = mdf[column].fillna(mdf[column].mode()[0])

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
X = mdf[["TAILPIPE", "_AGEG5YR", "SEX", "_RACE_G1", "_INCOMG", "_EDUCAG", "_BMI5CAT", 'POPEST2017_CIV']]
y = mdf['ASTHMA']


In [ ]:
clf = LogisticRegression()
clf.fit(X,y)
clf.coef_, clf.intercept_

In [ ]:
clf.score(X, y)

In [ ]:
np.exp(clf.coef_)

In [128]:
np.exp(clf.coef_[0][0]+1.96*clf.intercept_[0])

0.0006839292612615293

In [37]:
mdf[mdf['TAILPIPE']==1]['ASTHMA'].sum()/(mdf[mdf['TAILPIPE']==1]['ASTHMA'].count()-mdf[mdf['TAILPIPE']==1]['ASTHMA'].sum())

0.10281877960178415

In [38]:
mdf[mdf['TAILPIPE']==0]['ASTHMA'].sum()/(mdf[mdf['TAILPIPE']==0]['ASTHMA'].count()-mdf[mdf['TAILPIPE']==0]['ASTHMA'].sum())

0.09091087658096692

In [39]:
0.10281877960178415/0.09091087658096692

1.1309843603830156

In [22]:
0.1028/0.0909

1.130913091309131

In [140]:
clf.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [129]:
import statsmodels.api as sm

In [142]:
sm.Logit?

In [141]:
res = sm.Logit(y, X).fit()

Optimization terminated successfully.
         Current function value: 0.298059
         Iterations 7


In [137]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 ASTHMA   No. Observations:               747250
Model:                          Logit   Df Residuals:                   747243
Method:                           MLE   Df Model:                            6
Date:                Wed, 17 Nov 2021   Pseudo R-squ.:                -0.01807
Time:                        12:48:49   Log-Likelihood:            -2.2282e+05
converged:                       True   LL-Null:                   -2.1886e+05
Covariance Type:            nonrobust   LLR p-value:                     1.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
TAILPIPE       0.0665      0.009      7.220      0.000       0.048       0.085
_AGEG5YR      -0.0669      0.001    -62.322      0.000      -0.069      -0.065
SEX            0.0450      0.007      6.595      0.000       0.032       0.058
_RACE_G1      -0.2318      0.004    -52.281      0.000      -0.240      -0.223
_INCOMG       -0.1158      0.002    -63.502      0.000      -0.119      -0.112
_EDUCAG       -0.2881      0.004    -78.316      0.000      -0.295      -0.281
_BMI5CAT      -0.0608      0.004    -14.907      0.000      -0.069      -0.053
==============================================================================
"""

In [139]:
np.exp(res.params)

TAILPIPE    1.068785
_AGEG5YR    0.935322
SEX         1.046002
_RACE_G1    0.793119
_INCOMG     0.890637
_EDUCAG     0.749723
_BMI5CAT    0.941052
dtype: float64

In [160]:
params = res.params
conf = res.conf_int()
conf['Odds Ratio'] = params
conf.columns = ['5%', '95%', 'Odds Ratio']
a = np.exp(conf)

In [164]:
a.loc['TAILPIPE']

5%            1.082065
95%           1.122972
Odds Ratio    1.102329
Name: TAILPIPE, dtype: float64

In [157]:
def fit_logistic(X, y):
    res = sm.Logit(y, X).fit()
    params = res.params
    conf = res.conf_int()
    conf['Odds Ratio'] = params
    conf.columns = ['5%', '95%', 'Odds Ratio']
    print(np.exp(conf))

In [166]:
a = np.exp(conf).loc['TAILPIPE']

In [169]:
a

5%            1.082065
95%           1.122972
Odds Ratio    1.102329
Name: TAILPIPE, dtype: float64

In [158]:
carb_regions = mdf['EPA Region'].unique().tolist()
noncarb_regions = mdf['EPA Region'].unique().tolist()

for index, epa_region in enumerate([0,1,2,3,4,5,6,7,8,9,10]):
    if epa_region == 0 or (epa_region in carb_regions and epa_region in noncarb_regions):
        tdf = mdf[mdf['EPA Region'] == epa_region]
        X = tdf[["TAILPIPE", "_AGEG5YR", "SEX", "_RACE_G1", "_INCOMG", "_EDUCAG", "_BMI5CAT", 'POPEST2017_CIV']]
        y = tdf['ASTHMA']
        print(epa_region)
        fit_logistic(X, y)

0


ValueError: zero-size array to reduction operation maximum which has no identity